In [1]:
import os
os.getcwd()


'C:\\Users\\user\\RagChatbot'

In [3]:
# Step 5: Load and prepare the data
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer

# 1. Load the dataset
df = pd.read_csv("Training Dataset.csv")
df.head()



,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
# Convert each row into a single text string
docs = df.apply(
    lambda r: " | ".join(f"{c}: {r[c]}" for c in df.columns), axis=1
).tolist()

# Save for later use
pickle.dump(docs, open("docs.pkl", "wb"))
len(docs), "documents prepared"


(614, 'documents prepared')

In [5]:
import faiss
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Generate document embeddings
embeddings = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=True)

# Build a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save index for reuse
faiss.write_index(index, "faiss_index.bin")


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
# Step 8: Retrieval + Generation

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import faiss, pickle

# Load FAISS index & docs
index = faiss.read_index("faiss_index.bin")
docs = pickle.load(open("docs.pkl", "rb"))

# Load Flan‑T5 generator
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)

# Retrieval function
def retrieve(query, k=5):
    q_vec = embedder.encode([query], convert_to_numpy=True)
    _, I = index.search(q_vec, k)
    return [docs[i] for i in I[0]]

# Generation function
def generate_answer(query):
    context = "\n".join(retrieve(query))
    prompt = f"Context:\n{context}\n\nQ: {query}\nA:"
    return generator(prompt)[0]["generated_text"]

# Test it
print(generate_answer("Do married applicants get approved loans more often?"))


tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\user\RagChatbot\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. 

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


Married applicants are more likely to get approved loans than non-married applicants.


In [7]:
queries = [
    "What factors influence loan approval?",
    "What's the average loan amount for married couples?",
    "Do people with no credit history get loans?",
    "How does property area affect approval chances?"
]
for q in queries:
    print(f"Q: {q}")
    print("A:", generate_answer(q))
    print("-" * 40)


Q: What factors influence loan approval?
A: Gender, education, and occupation
----------------------------------------
Q: What's the average loan amount for married couples?
A: The average loan amount for married couples is 185.0.
----------------------------------------
Q: Do people with no credit history get loans?
A: No
----------------------------------------
Q: How does property area affect approval chances?
A: Rural areas have a higher approval rate than urban areas.
----------------------------------------
